In [30]:
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.techindicators import TechIndicators
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt


In [31]:
# stock data API alpha_vantage, documentation https://www.alphavantage.co/documentation/
key = 'A6QPO3VBTX5UHOAU'
# Chose your output format, or default to JSON (python dict)
ts = TimeSeries(key, output_format='pandas')
ti = TechIndicators(key)

ticker='AAPL'
# Get the data, returns a tuple
# aapl_data is a pandas dataframe, aapl_meta_data is a dict
aapl_data, aapl_meta_data = ts.get_intraday(symbol=ticker) #for intraday price, ts.get_intraday. Daily price, ts.get_daily
# aapl_sma is a dict, aapl_meta_sma also a dict
aapl_sma, aapl_meta_sma = ti.get_sma(symbol=ticker)

In [32]:
df=aapl_data
df.to_csv('priceData.csv')
#read data from the csv file
#df=pd.read_csv('priceData.csv').set_index(['date'])
df.head(15)

,1. open,2. high,3. low,4. close,5. volume
date,,,,,
2020-06-05 16:00:00,331.2100,331.8000,330.2000,331.6400,2317255.0
2020-06-05 15:45:00,331.4700,331.7500,331.1500,331.1800,1188071.0
2020-06-05 15:30:00,330.4400,331.5900,330.1026,331.4900,1197757.0
2020-06-05 15:15:00,330.2501,330.4700,329.8200,330.3900,622539.0
2020-06-05 15:00:00,330.1300,330.7100,330.0700,330.2450,615819.0
2020-06-05 14:45:00,330.0200,330.2400,329.2500,330.1500,849819.0
2020-06-05 14:30:00,329.8700,330.5500,329.7200,330.0200,635363.0
2020-06-05 14:15:00,330.3000,330.6900,329.7900,329.8510,636712.0
2020-06-05 14:00:00,330.8750,330.9900,330.1371,330.3100,527479.0


In [33]:
dfRatio = df[['1. open','2. high', '3. low','4. close']]/df['1. open'].iloc[-1]-1
dfRatio

,1. open,2. high,3. low,4. close
date,,,,
2020-06-05 16:00:00,0.028602,0.030435,0.025466,0.029938
2020-06-05 15:45:00,0.029410,0.030280,0.028416,0.028509
2020-06-05 15:30:00,0.026211,0.029783,0.025163,0.029472
2020-06-05 15:15:00,0.025621,0.026304,0.024286,0.026056
2020-06-05 15:00:00,0.025248,0.027050,0.025062,0.025606
...,...,...,...,...
2020-06-02 11:45:00,-0.008339,-0.004938,-0.009037,-0.005932
2020-06-02 11:30:00,-0.009161,-0.006491,-0.009534,-0.008337
2020-06-02 11:15:00,-0.004720,-0.004503,-0.009317,-0.009022


In [34]:
import plotly.graph_objects as go

dfChart = dfRatio


fig = go.Figure(data=[go.Candlestick(x=dfChart.index,
                open=dfChart['1. open'], high=dfChart['2. high'],
                low=dfChart['3. low'], close=dfChart['4. close'])
                     ])

#add xaxis_type='category' and fig.update_xaxes(autorange="reversed") to remove the datetime gaps
fig.update_layout(xaxis_type='category', xaxis_rangeslider_visible=False, title_text=ticker)
fig.update_xaxes(autorange="reversed")
fig.show()